In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import pandas as pd
import os
import glob
import sys
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
##loard validation data
val_data = pd.read_excel(r"C:/Users/Lenovo/Desktop/BCS/validation.xlsx")
val_data_feature = val_data.iloc[:,:10]
val_data_label = val_data.iloc[:,-1]

val_data_feature = np.array(val_data_feature)
val_data_label = np.array(val_data_label)

print(val_data_feature.shape)
print(val_data_label.shape)

In [ ]:
import torch.nn.init as init
class BP(nn.Module):
    def __init__(self, num_layers,hidden_size):
        super(BP,self).__init__()
        self.Liner1 = nn.Linear(10,hidden_size)
        self.queue= [nn.Sequential(nn.Linear(hidden_size, hidden_size), nn.LeakyReLU(0.1)) for _ in range(num_layers - 1)]
        self.model=nn.Sequential(*self.queue)
        self.Liner2 = nn.Linear(hidden_size,1)
        
        
    def forward(self,input):
        output = self.Liner1(input)
        output=self.model(output)
        output = self.Liner2(output)
        #output=output.squeeze(1)
        return output
n_epochs =50
bp=BP(7,12)
optimizer = torch.optim.Adam(bp.parameters(), lr=0.01)
MSEloss=nn.MSELoss()

In [ ]:
def val(val_data_feature, val_data_label,epoch_best):
    predict_list=[]
    bp.load_state_dict(torch.load(epoch_best))
    bp.eval()
    for val_data_feature in val_data_feature:
        val_data_feature = torch.from_numpy(val_data_feature).float()
        val_data_feature=val_data_feature.unsqueeze(0)
        predict=bp(val_data_feature)
        predict=predict.squeeze(1)
        predict=predict.squeeze(0)
        predict_list.append(predict.detach().numpy())
 
    predict_list=np.array(predict_list)
    loss = MSEloss(torch.from_numpy(predict_list),torch.from_numpy(val_data_label)).item()
    print("[R2:{}][loss:{}]".format(r2_score(val_data_label,predict_list), loss))
    return predict_list, loss

In [ ]:
predict_list, loss= val(val_data_feature,val_data_label,epoch_best="model_parameters_training.pth")